# Buffer Overflow

- one of the top 25 most dangerous software errors according to Common Weakness Enemuration (CWE) for several years: https://cwe.mitre.org/data/index.html - See CWE Top 25 - Out-of-bounds Write (CWE-787)
- learn buffer overflow with animation - [https://mirrorlearning.org/emate2/bufferoverflow/pub/bufferoverflow/Assets/index.html](https://mirrorlearning.org/emate2/bufferoverflow/pub/bufferoverflow/Assets/index.html)
- most Internet worms use buffer overflow vulnerabilities to propagate
- C and C++ assume that programmers are responsible for data integrity
- modern compilers and OSes provide various protections making it harder to exploit traditional buffer overflows
- once the buffer/memory is allocated, there's no built-in mechanism to safeguard the amount of data that can be stored into
    - this is called buffer overrun or buffer overflow
    - user data may overwrite other part of the memory it's not supposed to making the program behave differently or crashing it
    - attackers could take advantage of this flaw to control the program and may pawn the system by executing shellcode or payload of their choice
- buffer overflow can violate all three core principles of **CIA** triad - **Confidentiality, Integrity, and Availability**
- **Confidentiality**: allows attackers to read confidential or data that they're not supposed to have access to
- **Integrity**: allows attackers to modify part of the data/program that they're not supposed to
- **Availability**: allows attackes to crash the system (SEGFAULT) violating availbility of the service

- buffer can be overflown in all memory segments (Global, Stack and Heap)
- Stack overflow can have the most significant consequences!

## Disable ASLR and other compiler protections
- compile.sh script disables ASLR and compiler protections

In [1]:
# if value is 0; ASLR is disabled
! cat /proc/sys/kernel/randomize_va_space

0


In [2]:
# disable ASLR if randomize_va_space is not 0
! echo kali | sudo -S bash -c 'echo 0 | sudo -S tee /proc/sys/kernel/randomize_va_space'

[sudo] password for kali: 0


## Program's memory and stack may shift
- programs and data must be loaded into RAM inorder to execute
- depending on how the program is launched, several factors make the program's location in memory shift
    - even after virtual-address space layout randomization is disabled
- e.g. environment variables, length of program name, arguments to program, etc.
- gdb also sets its own environment variables such as LINES, COLUMNS, etc.
<img src="media/gdb-env.png">
- detail: https://stackoverflow.com/questions/17775186/buffer-overflow-works-in-gdb-but-not-without-it/17775966#17775966
- let's see `demos/env_demo.cpp` for demonstration

In [4]:
! cat demos/env_demo.cpp

#include <cstdio>
#include <stdlib.h> // getenv, putenv, setenv, unsetenv, etc.
// see: http://www.manpagez.com/man/3/getenv/

int main(int argc, char* argv[]) {
  if (argc == 2) {
    // read value of env variable
    //getenv(char * name) retruns NUL-terminated string; if name doesn't exists, returns NULL
    printf("%s=%s\n", argv[1], getenv(argv[1]));
  }
  else if (argc == 3) { 
    // set env variable, 0 doesn't overwrite, 1 overwrites
    int result = setenv(argv[1], argv[2], 0); // returns 0 on successful, -1 otherwise
    if (result == 0) {
      printf("Successfully set env variable!\n");
      printf("%s=%s\n", argv[1], getenv(argv[1]));
      // you can't use setenv to export variables from current process to the calling process (shell)
    }
    else {
      printf("Couldn't set env variable!");
    }
  }
  else {
    printf("%s\n", "Usage READ: program ENV_VAR_NAME");
    printf("%s\n", "Usage SET: program ENV_VAR_NAME VALUE");
  }
  return 0;
}


In [2]:
# let's compile the program as 32-bit
! g++ -m32 demos/env_demo.cpp -o env_demo.exe

In [3]:
# check the exe program
! ls -al env_demo.exe

-rwxr-xr-x 1 kali kali 15228 Feb 10 09:24 env_demo.exe


In [6]:
# run the program
! ./env_demo.exe

Usage READ: program ENV_VAR_NAME
Usage SET: program ENV_VAR_NAME VALUE


In [7]:
! ./env_demo.exe PATH

PATH=/home/kali/miniconda3/bin:/home/kali/miniconda3/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/local/games:/usr/games


In [8]:
! ./env_demo.exe USER

USER=kali


In [9]:
! ./env_demo.exe SOME_VAR

SOME_VAR=(null)


In [10]:
# set new env varirable for current projcess and it's children using fork
! ./env_demo.exe SOME_VAR "THIS IS THE VALUE FOR SOME_VAR"

Successfully set env variable!
SOME_VAR=THIS IS THE VALUE FOR SOME_VAR


## Observe difference between so_arg.cpp and so_stdio.cpp
- `demos/stack_overflow/so_stdio.cpp` doesn't take/use any argument
    - interactively prompts user to enter some text duing program execution
- `demos/stack_overflow/so_arg.cpp` takes argument
    - uses argument provided to the program
    - not an interactive program
- let's compile and run these programs to observe the address of buffer

In [3]:
! cat demos/stack_overflow/so_arg.cpp

#include <cstdio>
#include <cstring>
#include <iostream>

using namespace std;

#define BUFSIZE 128

void copydata(char *dst_buf, char *src_buf) {
    // similar to strcpy in cstdio
    size_t i = 0;
    // copy each character from src_buf to des_buffer
    // stop when NUL character is encountered
    while (*(src_buf+i) != 0) {
        *(dst_buf+i) = *(src_buf+i);
        i++;
    }
    *(dst_buf+i) = 0; // end with NUL character
}

void bad(char *data) {
    char buffer[BUFSIZE];
    printf("buffer is at %p\n", buffer);
    //strcpy(buffer, data);
    copydata(buffer, data);
    cout << "buffer contains: \n" << buffer << endl;
}

int main(int argc, char *argv[]) {
    bad(argv[1]);
    cout << "Good bye!" << endl;
    return 0;
}

- let's compile the program and run it
- use the provided compile.sh bash program
    - helps in compiling programs disabling all the overflow-related security controls

In [4]:
# let's see the script
! cat ./compile.sh

#! /bin/bash

# disable aslr
read aslr < /proc/sys/kernel/randomize_va_space
if (( $aslr != 0 )); then
    if (( id -u != 0)); then
        echo "Need sudo priviledge to disable aslr..."
    fi
    # provide password for sudo user
    ls /proc/sys/kernel/randomize_va_space
    echo 0 | sudo tee /proc/sys/kernel/randomize_va_space
fi

# compile disabling all the security flags
if (( $# == 1 )); then
    g++ -g -m32 -std=c++17 -fno-stack-protector -z execstack -no-pie $1 && echo "Output file is a.out"
elif (( $# == 2 )); then
    g++ -g -m32 -std=c++17 -fno-stack-protector -z execstack -no-pie -o $2 $1
else
    echo "Error... No input file provided..."
    echo "$0 <inputfile.cpp> [a.out]"
fi

In [5]:
! echo kali | sudo -S ./compile.sh demos/stack_overflow/so_arg.cpp so_arg.exe

[sudo] password for kali: 

In [6]:
# provide the argument to program
! ./so_arg.exe "Hello World!"
# Note the address of buffer!

buffer is at 0xffffc240
buffer contains: 
Hello World!
Good bye!


In [7]:
# provide different argument with the same length to program
! ./so_arg.exe "HELLO WORLD!"
# Note the address of buffer!

buffer is at 0xffffc240
buffer contains: 
HELLO WORLD!
Good bye!


In [8]:
! ./so_arg.exe $(python -c 'print("A"*100)')
# Note the address of buffer!

buffer is at 0xffffc1e0
buffer contains: 
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
Good bye!


In [9]:
! ./so_arg.exe $(python -c 'print("A"*200)')
# Note the address of buffer!

buffer is at 0xffffc180
buffer contains: 
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA


- Notice the different address for buffer in so_arg.exe based on the length of argument
- the address of the buffer shifts within the program
- this makes it a little difficult to exploit the program if the argument length varies

## buffer address in so_stdio.cpp
- compile and run the program and notice the address of buffer

In [10]:
! cat demos/stack_overflow/so_stdio.cpp

#include <stdio.h>
#include <string.h>
#include <sys/types.h>
#include <stdlib.h>
#include <unistd.h>
#include <iostream>
using namespace std;

#define BUFSIZE 128

using namespace std;

void give_shell(){
    // Set the gid to the effective gid
    // this prevents /bin/sh from dropping the privileges
    gid_t gid = getegid();
    setresgid(gid, gid, gid);
    system("/bin/sh");
}

char * mgets(char *dst) {
    char *ptr = dst;
    int ch; 
	/* skip leading white spaces */ 
    while ((ch = getchar()) && (ch == ' ' or ch == '\t') )
        ; 

    if ((ch == '\n') or (ch == EOF)) { 
        *ptr = '\0';
        return dst; 
    } 
    else
        *ptr = ch;

    /* now read the rest until \n or EOF */ 
    while (true) {
        ch = getchar();
        if (ch == '\n' or ch == EOF) break;
        *(++ptr) = ch; 
    }
    *(++ptr) = 0;
    return dst;
}

void bad() {
    char buffer[BUFSIZE];
    printf("buffer is at %p\n", buffer);
    cout << "Give me some text: ";
    fflush(stdou

In [11]:
! echo kali | sudo -S ./compile.sh demos/stack_overflow/so_stdio.cpp so_stdio.exe

[sudo] password for kali: 

In [12]:
# Must pipe the input or interactively type
! echo "Hello World" | ./so_stdio.exe 
# Note the address of buffer

buffer is at 0xffffc240
Give me some text: Acknowledged: Hello World with length 11
Good bye!


In [13]:
# run the progam with the same length argument of input
! echo "World_Hello" | ./so_stdio.exe
# Note the address of buffer; no change because same program is run with the same length of argument

buffer is at 0xffffc240
Give me some text: Acknowledged: World_Hello with length 11
Good bye!


In [14]:
# run the program with 100 As
! python -c 'print("A"*100)' | ./so_stdio.exe
# Note the address of buffer!

buffer is at 0xffffc240
Give me some text: Acknowledged: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA with length 100
Good bye!


In [15]:
# run the program with 200 As
! python -c 'print("A"*200)' | ./so_stdio.exe
# Note the address of buffer!

buffer is at 0xffffc240
Give me some text: Acknowledged: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA with length 200


#### Note:
- as demonstrated above, length of data read from standard input doesn't shift the addresses of the buffer/variables

## length of the program name
- let's give names of different lengths to the same program and notice the address of buffer being shifted with the program
- this is true regardless the program takes an argument or not 
    - NOTE the name of the program itself is passed as an argument to the program
    - the first argument to main's argv[] is the name of the program that gets pushed on to the main's stack

In [6]:
# let's compile the program and run the program with name stack_overflow_stdio.exe
! echo kali | sudo -S ./compile.sh demos/stack_overflow/so_stdio.cpp stack_overflow_stdio.exe

[sudo] password for kali: 

In [7]:
! echo "Hello_World" | ./stack_overflow_stdio.exe

buffer is at 0xffffc2a0
Give me some text: Acknowledged: Hello_World with length 11
Good bye!


In [9]:
# if the same program is run multiple times, 
! echo "Hello_Worl adfadsfasfasfasf asfadsfasdfadsfdfasfsa d" | ./stack_overflow_stdio.exe
# Notice no change in buffer address

buffer is at 0xffffc2a0
Give me some text: Acknowledged: Hello_Worl adfadsfasfasfasf asfadsfasdfadsfdfasfsa d with length 52
Good bye!


In [10]:
# let's compile nd run the program with name stack.exe
! echo kali | sudo -S ./compile.sh demos/stack_overflow/so_stdio.cpp stack.exe

[sudo] password for kali: 

In [11]:
! echo "Hello_World" | ./stack.exe

buffer is at 0xffffc2d0
Give me some text: Acknowledged: Hello_World with length 11
Good bye!


In [13]:
# let's compile and run the program with the name stack1.exe
! echo kali | sudo -S ./compile.sh demos/stack_overflow/so_stdio.cpp stack1.exe
# minor change in length may not shift it

[sudo] password for kali: 

In [14]:
! echo "Hello_World" | ./stack1.exe

buffer is at 0xffffc2d0
Give me some text: Acknowledged: Hello_World with length 11
Good bye!


In [15]:
# let's compile the program and run the program with the name stack11.exe
! echo kali | sudo -S ./compile.sh demos/stack_overflow/so_stdio.cpp stack11.exe

[sudo] password for kali: 

In [16]:
! echo "Hello_World" | ./stack11.exe
# perhaps, filename is not long engouh to make a difference

buffer is at 0xffffc2d0
Give me some text: Acknowledged: Hello_World with length 11
Good bye!


In [17]:
%%bash
# let's compile the program with output program name stack11111.exe
echo kali | sudo -S ./compile.sh demos/stack_overflow/so_stdio.cpp stack11111.exe
echo "Hello_World" | ./stack11111.exe

buffer is at 0xffffc2e0
Give me some text: Acknowledged: Hello_World with length 11
Good bye!


[sudo] password for kali: 

In [18]:
# let's find the no. of bytes the buffer address was shifted by
print(0xffffc2e0-0xffffc2d0)

16


## Run program with modified environment
- **env** command let's you run the program in a modified environment
- **env -i** - ignores environment or starts a program without one

In [19]:
! echo Hello | ./stack_overflow_stdio.exe

buffer is at 0xffffc2a0
Give me some text: Acknowledged: Hello with length 5
Good bye!


In [20]:
! echo Hello | env -i ./stack_overflow_stdio.exe

buffer is at 0xffffdd90
Give me some text: Acknowledged: Hello with length 5
Good bye!


In [21]:
# let's look at the current environment
! env

COLORFGBG=15;0
DBUS_SESSION_BUS_ADDRESS=unix:path=/run/user/1000/bus
DESKTOP_SESSION=lightdm-xsession
DISPLAY=:0.0
GDMSESSION=lightdm-xsession
HOME=/home/kali
LANG=en_US.utf8
LANGUAGE=
LOGNAME=kali
PANEL_GDK_CORE_DEVICE_EVENTS=0
PATH=/home/kali/miniconda3/bin:/home/kali/miniconda3/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/local/games:/usr/games
PWD=/home/kali/Sp21/EthicalHacking
QT_ACCESSIBILITY=1
QT_AUTO_SCREEN_SCALE_FACTOR=0
QT_QPA_PLATFORMTHEME=qt5ct
SESSION_MANAGER=local/K:@/tmp/.ICE-unix/852,unix/K:/tmp/.ICE-unix/852
SHELL=/usr/bin/zsh
SSH_AGENT_PID=895
SSH_AUTH_SOCK=/tmp/ssh-1ZCMbOabR0WB/agent.852
TERM=xterm-color
USER=kali
WINDOWID=0
XAUTHORITY=/home/kali/.Xauthority
XDG_CONFIG_DIRS=/etc/xdg
XDG_CURRENT_DESKTOP=XFCE
XDG_DATA_DIRS=/usr/share/xfce4:/usr/local/share/:/usr/share/:/usr/share
XDG_GREETER_DATA_DIR=/var/lib/lightdm/data/kali
XDG_MENU_PREFIX=xfce-
XDG_RUNTIME_DIR=/run/user/1000
XDG_SEAT=seat0
XDG_SEAT_PATH=/org/freedesktop/DisplayManager/

In [22]:
# print the difference in addresses
print(0xffffdd90-0xffffc290)

6912


In [25]:
! ./so_arg.exe Hello

buffer is at 0xffffc2d0
buffer contains: 
Hello
Good bye!


In [26]:
! env -i ./so_arg.exe Hello

buffer is at 0xffffdd90
buffer contains: 
Hello
Good bye!


In [27]:
# print the difference in addresses
print(0xffffdd90-0xffffc2c0)

6864


In [28]:
# argument length and program length still matters
# let's try with longer argument to demonstrate the shift in buffer's address
! env -i ./so_arg.exe "Hello there beautiful world!"

buffer is at 0xffffdd80
buffer contains: 
Hello there beautiful world!
Good bye!
